In [3]:
%load_ext autoreload
%autoreload 2

In [39]:
import sys
sys.path.insert(0, "../src")

import json
import random
import numpy as np
import scipy
from pathlib import Path
from collections import Counter, defaultdict
from joblib import Parallel, delayed

import constants
from gen.util import read_data, write_jsonl
from feverise import scifact, climatefever

In [12]:
feverp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/data/fever")
climatefdp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-climatefever")
scifactdp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-scifact")

modelp = Path("/scratch/users/k21190024/fact-check-transfer-learning/models")

# Generate Climate-FEVER Stratified 3-fold cross validation dataset

 - Fine-tune each fold with Climate-FEVER then validate on 1 fold Climate-FEVER and test on SciFact
 - 3-fold for computational reasons
 - Use same hyperparameter fine tuning parameters
 - This is done to reduce random effect from dataset

In [61]:
from sklearn.model_selection import StratifiedKFold

climatefd_folds_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-climatefever/paper_all_kfold")
kfold = StratifiedKFold(n_splits=3, random_state=constants.SEED, shuffle=True)

In [62]:
cf = read_data(climatefdp / "climatefever_paper_all.jsonl")
cf_idt = np.array([[d["id"], d["label"]] for d in cf])
cf_splits = {str(i): np.take(cf, test_idx) for i, (_, test_idx) in enumerate(kfold.split(cf_idt[:, 0], cf_idt[:, 1]))}

for i, fold in cf_splits.items():
    write_jsonl(climatefd_folds_p / f"fold{i}.jsonl", list(fold))

In [67]:
from collections import Counter

check_folds = {i: Counter([doc["label"] for doc in ls]) for i, ls in cf_splits.items()}
check_folds

{'0': Counter({'SUPPORTS': 218, 'REFUTES': 85, 'NOT ENOUGH INFO': 158}),
 '1': Counter({'SUPPORTS': 218, 'REFUTES': 84, 'NOT ENOUGH INFO': 158}),
 '2': Counter({'REFUTES': 84, 'SUPPORTS': 218, 'NOT ENOUGH INFO': 158})}